In [14]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Load the dataset
url = "https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv"
application_df = pd.read_csv(url)

In [15]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)


In [16]:
# Determine the number of unique values in each column.
unique_values = application_df.nunique()

# Display the unique values for each column
print(unique_values)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [17]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

# Display the counts of APPLICATION_TYPE
print(application_type_counts)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [18]:
# Choose a cutoff value and create a list of application types to be replaced
cutoff_value = 500  # This is an example cutoff value; adjust based on your analysis
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
binned_application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
print(binned_application_type_counts)

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64


In [19]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Display the counts of CLASSIFICATION
print(classification_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [20]:
# Look at CLASSIFICATION value counts greater than 1
classification_counts_gt1 = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() > 1]

# Display the counts of CLASSIFICATION greater than 1
print(classification_counts_gt1)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [21]:
# Choose a cutoff value and create a list of classifications to be replaced
cutoff_value = 100  # Example cutoff value, adjust based on your analysis
classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
binned_classification_counts = application_df['CLASSIFICATION'].value_counts()
print(binned_classification_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: count, dtype: int64


In [22]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies_df = pd.get_dummies(application_df)

# Display the first few rows of the new DataFrame with dummy variables
application_dummies_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,108590,1,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,1,5000,0,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
3,1,6692,1,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,142590,1,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,True,False


In [23]:
# Assuming 'IS_SUCCESSFUL' is the target column
y = application_dummies_df['IS_SUCCESSFUL'].values
X = application_dummies_df.drop(['IS_SUCCESSFUL'], axis=1).values

from sklearn.model_selection import train_test_split

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [64]:
#Optimizing Attempt 1 - Adding another hidden layer

# Initialize the model
nn_optimized = tf.keras.models.Sequential()

# First hidden layer
nn_optimized.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=X_train_scaled.shape[1]))

# Second hidden layer
nn_optimized.add(tf.keras.layers.Dense(units=30, activation='relu'))

# New third hidden layer - you can adjust the number of units and activation function as needed
nn_optimized.add(tf.keras.layers.Dense(units=20, activation='relu'))  # Example: adding a third hidden layer

# Output layer
nn_optimized.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model
nn_optimized.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = nn_optimized.fit(X_train_scaled, y_train, epochs=100, batch_size=128, validation_split=0.2, verbose=1)

# Check the structure of the model
nn_optimized.summary()


Epoch 1/100
172/172 [==============================] - 1s 4ms/step - loss: 0.5781 - accuracy: 0.7099 - val_loss: 0.5519 - val_accuracy: 0.7378
Epoch 2/100
172/172 [==============================] - 0s 3ms/step - loss: 0.5543 - accuracy: 0.7273 - val_loss: 0.5492 - val_accuracy: 0.7360
Epoch 3/100
172/172 [==============================] - 0s 3ms/step - loss: 0.5513 - accuracy: 0.7284 - val_loss: 0.5465 - val_accuracy: 0.7369
Epoch 4/100
172/172 [==============================] - 0s 3ms/step - loss: 0.5495 - accuracy: 0.7292 - val_loss: 0.5473 - val_accuracy: 0.7367
Epoch 5/100
172/172 [==============================] - 0s 3ms/step - loss: 0.5480 - accuracy: 0.7296 - val_loss: 0.5451 - val_accuracy: 0.7372
Epoch 6/100
172/172 [==============================] - 1s 3ms/step - loss: 0.5466 - accuracy: 0.7295 - val_loss: 0.5456 - val_accuracy: 0.7382
Epoch 7/100
172/172 [==============================] - 0s 3ms/step - loss: 0.5462 - accuracy: 0.7309 - val_loss: 0.5459 - val_accuracy: 0.7347

In [65]:
# Output the final accuracy for training and validation sets
training_accuracy = history.history['accuracy'][-1]
validation_accuracy = history.history['val_accuracy'][-1]

print(f"Final Training Accuracy: {training_accuracy*100:.2f}%")
print(f"Final Validation Accuracy: {validation_accuracy*100:.2f}%")

Final Training Accuracy: 73.81%
Final Validation Accuracy: 73.85%


In [66]:
from tensorflow.keras.optimizers import Adam

# Define a custom learning rate
optimizer = Adam(learning_rate=0.001)  # Example learning rate

# Compile the model with the custom optimizer
nn_optimized.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

history_2 = nn_optimized.fit(
    X_train_scaled,
    y_train,
    epochs=100,
    batch_size=128,
    validation_split=0.2,
    callbacks=[early_stopping],  # Include early stopping
    verbose=1
)

Epoch 1/100
172/172 [==============================] - 2s 4ms/step - loss: 0.5331 - accuracy: 0.7386 - val_loss: 0.5451 - val_accuracy: 0.7405
Epoch 2/100
172/172 [==============================] - 1s 3ms/step - loss: 0.5324 - accuracy: 0.7392 - val_loss: 0.5477 - val_accuracy: 0.7387
Epoch 3/100
172/172 [==============================] - 0s 3ms/step - loss: 0.5324 - accuracy: 0.7386 - val_loss: 0.5482 - val_accuracy: 0.7383
Epoch 4/100
172/172 [==============================] - 0s 3ms/step - loss: 0.5325 - accuracy: 0.7390 - val_loss: 0.5460 - val_accuracy: 0.7396
Epoch 5/100
172/172 [==============================] - 0s 3ms/step - loss: 0.5319 - accuracy: 0.7385 - val_loss: 0.5470 - val_accuracy: 0.7380
Epoch 6/100
172/172 [==============================] - 0s 3ms/step - loss: 0.5322 - accuracy: 0.7389 - val_loss: 0.5489 - val_accuracy: 0.7371
Epoch 7/100
172/172 [==============================] - 0s 3ms/step - loss: 0.5320 - accuracy: 0.7388 - val_loss: 0.5476 - val_accuracy: 0.7385

In [67]:
# Output the final accuracy for training and validation sets
training_accuracy = history_2.history['accuracy'][-1]
validation_accuracy = history_2.history['val_accuracy'][-1]

print(f"Final Training Accuracy: {training_accuracy*100:.2f}%")
print(f"Final Validation Accuracy: {validation_accuracy*100:.2f}%")

Final Training Accuracy: 73.91%
Final Validation Accuracy: 74.00%


In [ ]:
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

In [68]:
# Optimizing attempt 2 - Using differnet activation functions
from tensorflow.keras.layers import LeakyReLU

# Initialize the model
nn_optimized_2 = tf.keras.models.Sequential()

# First hidden layer
nn_optimized_2.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=X_train_scaled.shape[1]))

# Second hidden layer with 'tanh' activation
nn_optimized_2.add(tf.keras.layers.Dense(units=30, activation='tanh'))

# Third hidden layer with 'leakyrelu' activation
nn_optimized_2.add(tf.keras.layers.Dense(units=20))
nn_optimized_2.add(LeakyReLU(alpha=0.01))  # LeakyReLU needs to be added as a separate layer

# Output layer
nn_optimized_2.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model
nn_optimized_2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history_3 = nn_optimized_2.fit(X_train_scaled, y_train, epochs=100, batch_size=128, validation_split=0.2, verbose=1)

# Check the structure of the model
nn_optimized_2.summary()


Epoch 1/100
172/172 [==============================] - 2s 4ms/step - loss: 0.5793 - accuracy: 0.7076 - val_loss: 0.5467 - val_accuracy: 0.7391
Epoch 2/100
172/172 [==============================] - 0s 3ms/step - loss: 0.5552 - accuracy: 0.7277 - val_loss: 0.5451 - val_accuracy: 0.7398
Epoch 3/100
172/172 [==============================] - 0s 3ms/step - loss: 0.5502 - accuracy: 0.7303 - val_loss: 0.5423 - val_accuracy: 0.7414
Epoch 4/100
172/172 [==============================] - 0s 3ms/step - loss: 0.5483 - accuracy: 0.7314 - val_loss: 0.5459 - val_accuracy: 0.7338
Epoch 5/100
172/172 [==============================] - 1s 3ms/step - loss: 0.5473 - accuracy: 0.7296 - val_loss: 0.5419 - val_accuracy: 0.7414
Epoch 6/100
172/172 [==============================] - 1s 3ms/step - loss: 0.5460 - accuracy: 0.7323 - val_loss: 0.5413 - val_accuracy: 0.7423
Epoch 7/100
172/172 [==============================] - 0s 3ms/step - loss: 0.5457 - accuracy: 0.7312 - val_loss: 0.5407 - val_accuracy: 0.7383

In [69]:
# Output the final accuracy for training and validation sets
training_accuracy = history_3.history['accuracy'][-1]
validation_accuracy = history_3.history['val_accuracy'][-1]

print(f"Final Training Accuracy: {training_accuracy*100:.2f}%")
print(f"Final Validation Accuracy: {validation_accuracy*100:.2f}%")

Final Training Accuracy: 73.82%
Final Validation Accuracy: 73.72%


In [71]:
from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with the early stopping callback
history_4 = nn_optimized_2.fit(
    X_train_scaled,
    y_train,
    epochs=100,
    batch_size=128,
    validation_split=0.2,
    callbacks=[early_stopping],  # Include early stopping
    verbose=1
)

Epoch 1/100
172/172 [==============================] - 1s 3ms/step - loss: 0.5320 - accuracy: 0.7376 - val_loss: 0.5465 - val_accuracy: 0.7374
Epoch 2/100
172/172 [==============================] - 0s 3ms/step - loss: 0.5318 - accuracy: 0.7388 - val_loss: 0.5457 - val_accuracy: 0.7354
Epoch 3/100
172/172 [==============================] - 0s 3ms/step - loss: 0.5323 - accuracy: 0.7388 - val_loss: 0.5454 - val_accuracy: 0.7382
Epoch 4/100
172/172 [==============================] - 0s 3ms/step - loss: 0.5316 - accuracy: 0.7391 - val_loss: 0.5455 - val_accuracy: 0.7374
Epoch 5/100
172/172 [==============================] - 0s 3ms/step - loss: 0.5317 - accuracy: 0.7379 - val_loss: 0.5457 - val_accuracy: 0.7396
Epoch 6/100
172/172 [==============================] - 0s 3ms/step - loss: 0.5315 - accuracy: 0.7390 - val_loss: 0.5466 - val_accuracy: 0.7362
Epoch 7/100
172/172 [==============================] - 0s 3ms/step - loss: 0.5315 - accuracy: 0.7392 - val_loss: 0.5468 - val_accuracy: 0.7374

In [72]:
# Output the final accuracy for training and validation sets
training_accuracy = history_4.history['accuracy'][-1]
validation_accuracy = history_4.history['val_accuracy'][-1]

print(f"Final Training Accuracy: {training_accuracy*100:.2f}%")
print(f"Final Validation Accuracy: {validation_accuracy*100:.2f}%")

Final Training Accuracy: 73.80%
Final Validation Accuracy: 73.76%


In [73]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define model checkpoint callback
checkpoint = ModelCheckpoint(
    'best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max'
)

# Adjust the fit function to include the checkpoint
history_5 = nn_optimized_3.fit(
    X_train_scaled,
    y_train,
    epochs=100,
    batch_size=128,
    validation_split=0.2,
    callbacks=[early_stopping, checkpoint],  # Include both callbacks
    verbose=1
)

Epoch 1/100
172/172 [==============================] - 1s 3ms/step - loss: 0.5307 - accuracy: 0.7391 - val_loss: 0.5458 - val_accuracy: 0.7389
Epoch 2/100
 63/172 [=========>....................] - ETA: 0s - loss: 0.5273 - accuracy: 0.7398

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


172/172 [==============================] - 1s 3ms/step - loss: 0.5302 - accuracy: 0.7400 - val_loss: 0.5471 - val_accuracy: 0.7376
Epoch 3/100
172/172 [==============================] - 0s 3ms/step - loss: 0.5304 - accuracy: 0.7386 - val_loss: 0.5472 - val_accuracy: 0.7389
Epoch 4/100
172/172 [==============================] - 0s 3ms/step - loss: 0.5304 - accuracy: 0.7396 - val_loss: 0.5467 - val_accuracy: 0.7385
Epoch 5/100
172/172 [==============================] - 1s 3ms/step - loss: 0.5304 - accuracy: 0.7390 - val_loss: 0.5477 - val_accuracy: 0.7383
Epoch 6/100
172/172 [==============================] - 1s 4ms/step - loss: 0.5304 - accuracy: 0.7390 - val_loss: 0.5463 - val_accuracy: 0.7369
Epoch 7/100
172/172 [==============================] - 1s 4ms/step - loss: 0.5302 - accuracy: 0.7402 - val_loss: 0.5469 - val_accuracy: 0.7387
Epoch 8/100
172/172 [==============================] - 1s 4ms/step - loss: 0.5304 - accuracy: 0.7396 - val_loss: 0.5460 - val_accuracy: 0.7378
Epoch 9/100

In [74]:
# Output the final accuracy for training and validation sets
training_accuracy = history_5.history['accuracy'][-1]
validation_accuracy = history_5.history['val_accuracy'][-1]

print(f"Final Training Accuracy: {training_accuracy*100:.2f}%")
print(f"Final Validation Accuracy: {validation_accuracy*100:.2f}%")

Final Training Accuracy: 73.97%
Final Validation Accuracy: 73.92%


In [75]:
# Experiment with a smaller batch size
batch_size = 64  # Example of a smaller batch size

# Adjust the fit function to include the new batch size
history_6 = nn_optimized_3.fit(
    X_train_scaled,
    y_train,
    epochs=100,
    batch_size=batch_size,
    validation_split=0.2,
    callbacks=[early_stopping],  # Continue using early stopping
    verbose=1
)

Epoch 1/100
343/343 [==============================] - 1s 3ms/step - loss: 0.5341 - accuracy: 0.7379 - val_loss: 0.5469 - val_accuracy: 0.7323
Epoch 2/100
343/343 [==============================] - 1s 2ms/step - loss: 0.5334 - accuracy: 0.7379 - val_loss: 0.5468 - val_accuracy: 0.7374
Epoch 3/100
343/343 [==============================] - 1s 2ms/step - loss: 0.5327 - accuracy: 0.7365 - val_loss: 0.5454 - val_accuracy: 0.7383
Epoch 4/100
343/343 [==============================] - 1s 2ms/step - loss: 0.5324 - accuracy: 0.7387 - val_loss: 0.5444 - val_accuracy: 0.7392
Epoch 5/100
343/343 [==============================] - 1s 3ms/step - loss: 0.5329 - accuracy: 0.7372 - val_loss: 0.5470 - val_accuracy: 0.7356
Epoch 6/100
343/343 [==============================] - 1s 3ms/step - loss: 0.5326 - accuracy: 0.7393 - val_loss: 0.5470 - val_accuracy: 0.7372
Epoch 7/100
343/343 [==============================] - 1s 3ms/step - loss: 0.5322 - accuracy: 0.7369 - val_loss: 0.5479 - val_accuracy: 0.7374

In [76]:
# Output the final accuracy for training and validation sets
training_accuracy = history_6.history['accuracy'][-1]
validation_accuracy = history_6.history['val_accuracy'][-1]

print(f"Final Training Accuracy: {training_accuracy*100:.2f}%")
print(f"Final Validation Accuracy: {validation_accuracy*100:.2f}%")

Final Training Accuracy: 73.77%
Final Validation Accuracy: 73.78%


In [77]:
# Save the optimized model to an HDF5 file
nn_optimized.save("AlphabetSoupCharity_Optimization.h5")

In [78]:
from google.colab import files
files.download("AlphabetSoupCharity_Optimization.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>